# Tutorial 2: Reverse non-equilibrium simulation of a MPCD fluid

In this part of the tutorials, we will use the azplugins implementation of the [reverse perturbation method](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.59.4894) by M&uuml;ller-Plathe to generate shear flow in a simple multi-particle collision dynamics (MPCD) fluid. For details on the general method and setup, refer to the [LJ reverse perturbation tutorial](01_reverse_perturbation_1_LJ.ipynb).

### System setup

First, we need to set up a simple MPCD simulation.

```Python
import numpy as np
import sys
import hoomd
from hoomd import md
from hoomd import data
from hoomd import azplugins
from hoomd.azplugins import mpcd

L = 10
kT = 1.0
rho_mpcd = 5
viscosity_mpcd = 3.955

hoomd.context.initialize()
hoomd.context.SimulationContext()

snapshot = hoomd.data.make_snapshot(N=0,box=data.boxdim(L=L))
system = hoomd.init.read_snapshot(snapshot)

N_mpcd = int(rho_mpcd*system.box.get_volume())
snap = hoomd.mpcd.data.make_snapshot(N=N_mpcd)

snap.particles.position[:] = np.random.uniform(-L/2.0, L/2.0, (N_mpcd,3))
snap.particles.velocity[:] = np.random.normal(0, np.sqrt(kT), (N_mpcd,3))
snap.particles.velocity[:] -= np.average(snap.particles.velocity,axis=0)

mpcd_sys = hoomd.mpcd.init.read_snapshot(snap)
```

We want to set up a simulation at a reduced temperature of $k_\text{B}T=1.0$ and at a number density of $\rho_\text{mpcd}=5$ in a cubic box with $L = 10$. Both positions and velocities are set up randomly in the box. If you have an idea what flow field to expect, you can make equilibration faster by modifying the velocities accordingly. MPCD kinematic viscosities can be calculated from (see this [publication](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.72.016701)): 

$\eta = \eta_\text{coll} +\eta_\text{kin}$

$\frac{\eta_\text{coll}}{\sqrt{k_\text{B}T a^2/m}} = \frac{1}{\lambda}\frac{1-\cos{\alpha}}{18}\left(1 -\frac{1}{\rho_\text{mpcd}}\right)$

$\frac{\eta_\text{coll}}{\sqrt{k_\text{B}T a^2/m}} = \lambda\left(\frac{1}{(4-2\cos{\alpha}-2\cos{2\alpha})} \frac{5\rho_\text{mpcd}}{\rho_\text{mpcd}-1} -\frac{1}{2}\right)$

where $m=1$ is the mass of each particle, $a=1$ is the grid size, $\alpha$ is the collision angle, and $\lambda$ is the timestep, defined later. We use the same setup to measure fluid properties as in the first half of the tutorial. 

### Eqilibration 

For the tutorial the system was equilibrated for 10,000 timesteps with  stochastic rotation dynamics (SRD) with an angle $\alpha$= ``angle=130`` at a timestep of $\lambda$= ``dt=0.1``. For a real simulation it may be neccessary to equilibrate the system properly/longer before starting the shear flow. More information about the MPCD algorithm and how to couple MPCD fluids to MD particles can be found in the [hoomd-blue doumentation](https://hoomd-blue.readthedocs.io/en/stable/package-mpcd.html).

```Python
hoomd.mpcd.integrator(dt=0.1)
mpcd_sys.sorter.set_period(25)
srd   = hoomd.mpcd.collide.srd(seed=512, period=1, angle=130., kT=kT)
bulk  = hoomd.mpcd.stream.bulk(period=1)

hoomd.run(1e5)
```

The main difference between the LJ reverse perturbation and the MPCD reverse perturbation is a slight change in the function and its arguments. No group argment is needed, but all the other parameters are the same. The setup for measuring quantites is the same as for the LJ fluid, except that the mpcd doesn't have a gsd snapshot functionality. 

```Python

f = azplugins.mpcd.reverse_perturbation(width=1,Nswap=1,period=1,target_momentum=0.5)

# log the exchanged momentum during the simulation 
log = hoomd.analyze.log(filename="tutorial_reverse_perturbation_2_mpcd.log",
                        quantities=['rp_momentum'],
                        period=1e2,overwrite=True)

measure_vel = azplugins.flow.FlowProfiler(system=system, axis=2, bins=10, range=(-L/2,L/2), area=L**2)
analyze = hoomd.analyze.callback(measure_vel, period=1e2)

```

### Run the non-equilibrium simulation 

```Python
hoomd.run(1e6)

snap = mpcd_sys.take_snapshot()
pos  = snap.particles.position
vel  = snap.particles.velocity
np.save('tutorial_reverse_perturbation_2_mpcd_pos.npy',pos)
np.save('tutorial_reverse_perturbation_2_mpcd_vel.npy',vel)
np.savetxt('tutorial_reverse_perturbation_2_mpcd_vx.hist', np.column_stack((measure_vel.centers,  measure_vel.number_velocity[:,0])))
```


For MPCD it is impractical to write actual snapshots of the system. Instead, configuration is saved at the end in numpy arrays for restarting purposes, which can be read in for the next simulation instead of random positions/velocities as starting point.

### azplugins.flow.reverse_perturbation parameters  

Please see the [LJ reverse perturbation tutorial](01_reverse_perturbation_1_LJ.ipynb) for a detailed description of the parameters. 

<div class="alert alert-info">

Note

For MPCD, the speed of sound should be the upper limit for velocites, so it is advisable to not exceed $v_\text{max}=0.5$ in the desired flow field. 

</div>

Because the viscosity is known, $v_\text{max}$ can be computed from the target Reynolds number $Re$: 

$v_\text{max} = \frac{2\eta_\text{mpcd}Re}{\rho_\text{mpcd}L_z}$. 

You can also estimate $v_\text{max}$ from the known viscosity and the reverse_perturbation parameters:

$v_\text{max}=\frac{\text{target momentum}\cdot N_\text{swap}/\text{period}}{L_x L_y\lambda\eta_\text{mpcd}}\frac{L_z}{4}$.

This can also be useful to estimate the parameters for the reverse perturbation based on a desired Reynolds number or for setting up the velocity profile close to the expected shape at the beginning of the simulation. 

### Analyzing the results 

The simulation can be analyzed the same way than the LJ fluid. Because the mpcd viscosities are known, the pure mpcd fluid can be a useful check for the simulation setup.

### Future reading

- Original publication: Florian Mueller-Plathe. Reversing the perturbation innonequilibrium
    molecular dynamics:  An easy way to calculate the shear viscosity of
    fluids. Phys. Rev. E, 59:4894-4898, May 1999.
    <http://dx.doi.org/10.1103/PhysRevE.59.4894>
- Follow-up publications, describe the algorithm in more detail: 
    * Müller-Plathe, F., & Bordat, P. (2004). Reverse Non-equilibrium Molecular Dynamics. Lecture Notes in Physics, 310–326. <http://dx.doi.org/10.1007/978-3-540-39895-0_10>
    * Müller-Plathe, F., & Reith, D. (1999). Cause and effect reversed in non-equilibrium molecular dynamics: an easy route to transport coefficients. Computational and Theoretical Polymer Science, 9(3-4), 203–209. <http://dx.doi.org/10.1016/s1089-3156(99)00006-9>
- Our own publications about the method, describe some of the observed problems of the algorithm: 
    * Instability of Shear Flows in Spatially Periodic Domains, MP Howard, A Statt, HA Stone, TM Truskett, arXiv preprint arXiv:1907.07086 <https://arxiv.org/pdf/1907.07086.pdf>
    * Unexpected secondary flows in reverse nonequilibrium shear flow simulations, A Statt, MP Howard, AZ Panagiotopoulos, Physical Review Fluids 4 (4), 043905 <https://arxiv.org/pdf/1811.04097.pdf>
- This is the documentation page of the lammps implementation of the same algorithm: https://lammps.sandia.gov/doc/fix_viscosity.html Comparing against a different simulation package can be a useful for finding bugs.